
영어: https://towardsdatascience.com/classification-with-pycaret-a-better-machine-learning-library-cff07a10a28c


블로그: https://medium.com/@john_analyst/pycaret-%EC%BD%94%EB%93%9C-%EB%AA%87-%EC%A4%84%EB%A1%9C-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-18a8de4b6024

github 자료: https://github.com/doorBW/kaggle_practice/blob/master/titanic/80pcts_solution_180603/Beomwoo_Titanic_solution.ipynb
        
        
리더보드: https://www.kaggle.com/c/titanic/leaderboard

In [1]:
import pandas as pd

df = pd.read_csv(r'./train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Drop irrelevant columns (Ticket and PassengerId)

In [2]:
df.drop(['Ticket', 'PassengerId'], axis=1, inplace=True)

Remap Sex column to zeros and ones

In [3]:
gender_mapper = {'male': 0, 'female': 1} 
df['Sex'].replace(gender_mapper, inplace=True)

Check if a passenger had a unique title (like doctor) or had something more generic (like Mr., Miss.)

— can be extracted from the Name column

In [4]:
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].strip().split(' ')[0]) 
df['Title'] = [0 if x in ['Mr.', 'Miss.', 'Mrs.'] else 1 for x in df['Title']] 
df = df.rename(columns={'Title': 'Title_Unusual'}) 
df.drop('Name', axis=1, inplace=True)

Check if cabin information was known — if the value of Cabin column is not NaN

In [5]:
df['Cabin_Known'] = [0 if str(x) == 'nan' else 1 for x in df['Cabin']] 
df.drop('Cabin', axis=1, inplace=True)

Create dummy variables from the Embarked column — 3 options

In [6]:
emb_dummies = pd.get_dummies(df['Embarked'], drop_first=True, prefix='Embarked') 
df = pd.concat([df, emb_dummies], axis=1) 
df.drop('Embarked', axis=1, inplace=True) 

Fill Age values with the simple mean

In [7]:
df['Age'] = df['Age'].fillna(int(df['Age'].mean()))

EDA result

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Title_Unusual,Cabin_Known,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,0,0,0,1
1,1,1,1,38.0,1,0,71.2833,0,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,1,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,0,1


## Model selection and training

In [9]:
from pycaret.classification import * 

clf = setup(df, target='Survived', session_id=42)

,Description,Value
0,session_id,42
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(891, 11)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [10]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8363,0.8588,0.6795,0.8506,0.7531,0.6333,0.6440,0.6670
gbc,Gradient Boosting Classifier,0.8331,0.8571,0.6882,0.8363,0.7525,0.6287,0.6374,0.0160
lr,Logistic Regression,0.8282,0.8511,0.7274,0.7922,0.7575,0.6251,0.6271,0.4750
ridge,Ridge Classifier,0.8218,0.0000,0.7275,0.7763,0.7505,0.6123,0.6136,0.0050
lda,Linear Discriminant Analysis,0.8186,0.8526,0.7275,0.7696,0.7472,0.6061,0.6074,0.0060
lightgbm,Light Gradient Boosting Machine,0.8123,0.8534,0.7060,0.7698,0.7347,0.5902,0.5932,0.0120
ada,Ada Boost Classifier,0.8105,0.8395,0.7406,0.7471,0.7427,0.5929,0.5940,0.0150
xgboost,Extreme Gradient Boosting,0.8059,0.8551,0.7190,0.7531,0.7329,0.5810,0.5840,0.2390
rf,Random Forest Classifier,0.8042,0.8450,0.7100,0.7568,0.7282,0.5761,0.5806,0.0380
et,Extra Trees Classifier,0.7834,0.8164,0.6926,0.7167,0.7015,0.5321,0.5348,0.0330


In [31]:
model_cat = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8254,0.8045,0.7083,0.8095,0.7556,0.6207,0.6240
1,0.8095,0.8092,0.6087,0.8235,0.7000,0.5650,0.5789
2,0.8889,0.9207,0.8261,0.8636,0.8444,0.7581,0.7585
3,0.9355,0.9019,0.8261,1.0000,0.9048,0.8566,0.8656
4,0.8226,0.8384,0.5652,0.9286,0.7027,0.5867,0.6234
5,0.7097,0.7430,0.5217,0.6316,0.5714,0.3549,0.3586
6,0.8387,0.8584,0.6522,0.8824,0.7500,0.6349,0.6507
7,0.7903,0.8874,0.6087,0.7778,0.6829,0.5298,0.5386
8,0.8065,0.8484,0.6522,0.7895,0.7143,0.5699,0.5759
9,0.9355,0.9766,0.8261,1.0000,0.9048,0.8566,0.8656


In [33]:
model_gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8095,0.7799,0.7083,0.7727,0.7391,0.5896,0.5910
1,0.7937,0.8516,0.6087,0.7778,0.6829,0.5333,0.5421
2,0.9048,0.9065,0.8261,0.9048,0.8636,0.7907,0.7926
3,0.9194,0.9208,0.7826,1.0000,0.8780,0.8191,0.8329
4,0.8065,0.8344,0.6087,0.8235,0.7000,0.5618,0.5758
5,0.7742,0.7302,0.5217,0.8000,0.6316,0.4790,0.5017
6,0.8387,0.8763,0.6522,0.8824,0.7500,0.6349,0.6507
7,0.7903,0.8852,0.6957,0.7273,0.7111,0.5467,0.5470
8,0.7903,0.8094,0.6957,0.7273,0.7111,0.5467,0.5470
9,0.9032,0.9766,0.7826,0.9474,0.8571,0.7850,0.7932


## 모델 튜닝

꼭 튜닝을 한다고 성능이 좋아지는건 아님.

In [32]:
#오류
#https://github.com/pycaret/pycaret/issues/516
#tuned_model = tune_model('catboost')

tuned_cat = tune_model(model_cat)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7937,0.7938,0.5833,0.8235,0.6829,0.5365,0.5540
1,0.8095,0.7897,0.6087,0.8235,0.7000,0.5650,0.5789
2,0.9206,0.9196,0.8261,0.9500,0.8837,0.8239,0.8286
3,0.9355,0.9075,0.8261,1.0000,0.9048,0.8566,0.8656
4,0.8226,0.8016,0.5652,0.9286,0.7027,0.5867,0.6234
5,0.7742,0.7586,0.5217,0.8000,0.6316,0.4790,0.5017
6,0.8226,0.8361,0.6087,0.8750,0.7179,0.5945,0.6154
7,0.7742,0.8779,0.5652,0.7647,0.6500,0.4888,0.5010
8,0.8065,0.8339,0.6087,0.8235,0.7000,0.5618,0.5758
9,0.9032,0.9710,0.7391,1.0000,0.8500,0.7809,0.8004


In [34]:
tuned_gbc = tune_model(model_gbc)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7937,0.8018,0.6250,0.7895,0.6977,0.5442,0.5528
1,0.7778,0.7951,0.6522,0.7143,0.6818,0.5116,0.5129
2,0.9206,0.9087,0.8261,0.9500,0.8837,0.8239,0.8286
3,0.9032,0.9231,0.7826,0.9474,0.8571,0.7850,0.7932
4,0.8226,0.8166,0.6522,0.8333,0.7317,0.6021,0.6122
5,0.7419,0.7581,0.5217,0.7059,0.6000,0.4158,0.4261
6,0.8226,0.8445,0.6087,0.8750,0.7179,0.5945,0.6154
7,0.7419,0.8634,0.6087,0.6667,0.6364,0.4370,0.4381
8,0.7903,0.8244,0.6957,0.7273,0.7111,0.5467,0.5470
9,0.8871,0.9749,0.7391,0.9444,0.8293,0.7468,0.7593


## 예측

In [36]:
test= pd.read_csv(r'./test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [37]:
test.drop(['Ticket', 'PassengerId'], axis=1, inplace=True) 

gender_mapper = {'male': 0, 'female': 1} 
test['Sex'].replace(gender_mapper, inplace=True) 

test['Title'] = test['Name'].apply(lambda x: x.split(',')[1].strip().split(' ')[0]) 
test['Title'] = [0 if x in ['Mr.', 'Miss.', 'Mrs.'] else 1 for x in test['Title']] 
test = test.rename(columns={'Title': 'Title_Unusual'}) 
test.drop('Name', axis=1, inplace=True) 

test['Cabin_Known'] = [0 if str(x) == 'nan' else 1 for x in test['Cabin']] 
test.drop('Cabin', axis=1, inplace=True) 

emb_dummies = pd.get_dummies(test['Embarked'], drop_first=True, prefix='Embarked') 
test = pd.concat([test, emb_dummies], axis=1) 
test.drop('Embarked', axis=1, inplace=True) 

test['Age'] = test['Age'].fillna(int(test['Age'].mean()))

test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Title_Unusual,Cabin_Known,Embarked_Q,Embarked_S
0,3,0,34.5,0,0,7.8292,0,0,1,0
1,3,1,47.0,1,0,7.0000,0,0,0,1
2,2,0,62.0,0,0,9.6875,0,0,1,0
3,3,0,27.0,0,0,8.6625,0,0,0,1
4,3,1,22.0,1,1,12.2875,0,0,0,1


In [49]:
cat_final = finalize_model(tuned_cat)
predict_final = predict_model(cat_final, test)

In [50]:
predict_final.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Title_Unusual,Cabin_Known,Embarked_Q,Embarked_S,Label,Score
0,3,0,34.5,0,0,7.8292,0,0,1,0,0,0.8694
1,3,1,47.0,1,0,7.0000,0,0,0,1,0,0.6232
2,2,0,62.0,0,0,9.6875,0,0,1,0,0,0.8970
3,3,0,27.0,0,0,8.6625,0,0,0,1,0,0.8771
4,3,1,22.0,1,1,12.2875,0,0,0,1,0,0.5460


In [51]:
ID_list = pd.read_csv(r'./test.csv')['PassengerId']

submission = pd.DataFrame({
    "PassengerId": ID_list,
    "Survived": predict_final['Label']
})

submission.to_csv('submission_test1.csv', index=False)
submission = pd.read_csv(r'./submission_test1.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
